In [30]:
from aneurysm_utils import postprocessing
case_list_shuffled=["A130_R","A118","A120","A115","A133","A073","A072","A077","A064"]
mri_imgs_test=[]

for cases in case_list_shuffled:
    file_path=f"../../../../../data/training/{cases}_orig.nii.gz"
    mri_imgs_test.append(nib.load(file_path).get_fdata())
mri_imgs_test=postprocessing.resample(mri_imgs_test,(70,70,60))
mri_imgs_test=preprocessing.preprocess(env,mri_imgs_test,preprocessing_params)

[INFO] Preprocessing: Min Max Normalize...
[INFO] Preprocessing: Intensity Segmentation...


In [31]:
from aneurysm_utils import postprocessing
case_list_shuffled=["A130_R","A118","A120","A115","A133","A073","A072","A077","A064"]
labels_test=[]

for cases in case_list_shuffled:
    file_path=f"../../../../../data/training/{cases}_masks.nii.gz"
    labels_test.append(nib.load(file_path).get_fdata())
labels_test=postprocessing.resample(labels_test,(70,70,60))
#labels_test=preprocessing.preprocess(env,labels_test,preprocessing_params)

In [32]:
from aneurysm_utils.utils.pytorch_utils import predict
from aneurysm_utils.models.unet_3d_oktay import unet_3D
from aneurysm_utils.models.attention_unet import unet_grid_attention_3D
from aneurysm_utils.models.pointnet  import SegNet 

In [33]:

model = SegNet(dropout=0.38, num_classes=2, sample_rate1 =0.2263, sample_rate2 =0.2941,start_radius =7.254)
PATH = "/group/cake/our-git-project/models/2021-07-14-20-52-20_mask-pytorch-segnet/SegNet_SegNet_3000.pt"
device = torch.device('cpu')
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

In [34]:
from aneurysm_utils.utils import pytorch_utils
from torch.utils.data.dataloader import DataLoader
#datasets_folder = exp._env.project_folder
test_dataset = pytorch_utils.PyTorchGeometricDataset(
                mri_images=mri_imgs_test,
                labels=labels_test,
                root="/group/cake/our-git-project/processed",
                split="test",
                force_processing=True            
            )

del mri_imgs_test,labels_test

Processing...
Done!
/group/cake/our-git-project/processed/processed/test.pt


In [35]:
from torch_geometric.data import DataLoader as DataLoaderGeometric
test_loader = DataLoaderGeometric(
            test_dataset,
            batch_size=1,  # TODO: use fixed batch size of 5
            shuffle=False,
            num_workers=0,
            pin_memory=True,
        )
predictions = predict(model, test_loader, apply_softmax=False,ispointnet=True,dimension=(70, 70, 60))

In [39]:
converter_probs = map(lambda x: x[0], predictions)
probs = list(converter_probs)

In [40]:
for n in range(len(predictions)):
    print(probs[n].shape)

(70, 70, 60)
(70, 70, 60)
(70, 70, 60)
(70, 70, 60)
(70, 70, 60)
(70, 70, 60)
(70, 70, 60)
(70, 70, 60)
(70, 70, 60)


In [42]:

###############   Name und Ort gegebenfalls ändern
np.save("/group/cake/our-git-project/develop/Leo/pointnet_3.npy",probs)